In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!gdown '18KAfv6YSdmFGOhzB9G2ZyYoyMGqlHzv2'

Downloading...
From: https://drive.google.com/uc?id=18KAfv6YSdmFGOhzB9G2ZyYoyMGqlHzv2
To: /kaggle/working/validation_논문.zip
100%|██████████████████████████████████████| 20.9M/20.9M [00:00<00:00, 78.5MB/s]


In [2]:
import os
print(os.listdir("/kaggle/working"))

['.virtual_documents', 'validation_논문.zip']


In [3]:
import zipfile

# 파일 경로 설정
zip_path = '/kaggle/working/validation_논문.zip'
directory_to_extract = '/kaggle/working/'

# 압축 해제
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for file in zip_ref.namelist():
        # 파일 이름 복구 (cp437에서 euc-kr로 변환)
        decoded_file_name = file.encode('cp437').decode('euc-kr')

        # 압축 해제 후, 복구된 파일명으로 저장
        zip_ref.extract(file, directory_to_extract)

        # 파일 경로 수정
        original_path = os.path.join(directory_to_extract, file)
        new_path = os.path.join(directory_to_extract, decoded_file_name)

        # 파일 이름 변경 (복구된 파일명으로)
        if os.path.exists(original_path):
            os.rename(original_path, new_path)

In [4]:
import shutil
shutil.rmtree('/kaggle/working/validation_│φ╣«')

In [5]:
os.listdir('/kaggle/working/validation_논문')

['논문요약20231006_Validation.json']

In [6]:
j = ['논문요약20231006_Validation.json']

In [7]:
import json
import pandas as pd
# 빈 리스트 초기화
original_text = []  # 원문 리스트
original_summary = []  # 원문 요약 리스트
section_text = []  # 섹션 원문 리스트
section_summary = []  # 섹션 요약 리스트

# 파일 리스트 j (파일 리스트를 정의해야 합니다)
for file in j:
    with open(f'/kaggle/working/validation_논문/{file}', 'r', encoding='utf-8') as f:
        data = json.load(f)

        # 데이터에서 각 항목 추출
        for i in range(len(data)):
            # 전체 원문과 요약
            df = pd.DataFrame(data[i]['data'])

            # 전체 원문과 전체 요약 추출
            df['original_text'] = df['summary_entire'].apply(lambda x: x[0]['orginal_text'])  # 전체 원문
            df['original_summary'] = df['summary_entire'].apply(lambda x: x[0]['summary_text'])  # 원문 요약

            # 섹션 원문과 섹션 요약
            df['section_text'] = df['summary_section'].apply(lambda x: x[0]['orginal_text'])  # 섹션 원문
            df['section_summary'] = df['summary_section'].apply(lambda x: x[0]['summary_text'])  # 섹션 요약

            # 원문과 원문 요약을 리스트에 추가
            original_text.extend(df['original_text'].values)
            original_summary.extend(df['original_summary'].values)

            # 섹션 원문과 섹션 요약을 리스트에 추가
            section_text.extend(df['section_text'].values)
            section_summary.extend(df['section_summary'].values)

# 1. 데이터프레임에 source_type 추가
original_df = pd.DataFrame({
    'original_text': original_text,
    'summary_text': original_summary,
    'source_type': ['original'] * len(original_text)
})
section_df = pd.DataFrame({
    'original_text': section_text,
    'summary_text': section_summary,
    'source_type': ['section'] * len(section_text)
})

# 2. 통합 및 셔플
merged_df = pd.concat([original_df, section_df]).sample(frac=1).reset_index(drop=True)

# 3. 프롬프트 추가
def add_prompt(row):
    if row["source_type"] == "original":
        return "[전체요약] " + row["original_text"]
    else:
        return "[섹션요약] " + row["original_text"]

merged_df["input_text"] = merged_df.apply(add_prompt, axis=1)

In [8]:
merged_df

,original_text,summary_text,source_type,input_text
0,"대학에서 시간강사는 교육과 연구를 통해 유능한 인재 배출, 연구 경쟁력 강화, 그리...","대학에서 시간강사는 교육과 연구를 통해 유능한 인재 배출, 연구 경쟁력 강화, 그리...",original,"[전체요약] 대학에서 시간강사는 교육과 연구를 통해 유능한 인재 배출, 연구 경쟁력..."
1,이 연구는 사회주의기업책임관리제가 지난 20년간 지속된 노동자의 무보수노동에 대해 ...,20년간 노동자의 무보수노동과 노동행위자의 시각에서 사회주의기업책임관리제 관리자의 ...,original,[전체요약] 이 연구는 사회주의기업책임관리제가 지난 20년간 지속된 노동자의 무보수...
2,"노래에서 가사와 음악의 관계는 항상 흥미롭다. 언어와 음악이라는, 닮았지만 다른 두...","작곡가가 자신의 의도를 표현하기 위하여 언어의 특성, 음악의 성질을 잘 사용할 뿐 ...",section,"[섹션요약] 노래에서 가사와 음악의 관계는 항상 흥미롭다. 언어와 음악이라는, 닮았..."
3,글로벌 시대에 인적자원의 질과 경쟁력을 높이기 위해 고등평생학습 증진을 위한 정책적...,인적자원의 질과 경쟁력을 높이기 위해 고등평생학습 증진을 위한 정책적 노력이 지속되...,original,[전체요약] 글로벌 시대에 인적자원의 질과 경쟁력을 높이기 위해 고등평생학습 증진을...
4,"운동선수의 은퇴는 일반인과 비교하면 비교적 이른 시기인 20~30대에 이루어지며, ...","대학 야구선수들은 졸업이 다가올수록 은퇴에 대한 불안감이 높아진다. 한편, 진로 태...",section,[섹션요약] 운동선수의 은퇴는 일반인과 비교하면 비교적 이른 시기인 20~30대에 ...
...,...,...,...,...
35995,정부신뢰의 하락 이유에 대해 McAllister(1999)는 민주적 가치에 대한 시...,정부신뢰의 하락 이유에 대해 민주적 가치에 대한 시민의 열망을 정부제도와 정치지도자...,section,[섹션요약] 정부신뢰의 하락 이유에 대해 McAllister(1999)는 민주적 가...
35996,현재 바이폴러만의 프로세스(bipolar only process)로 사용되는 전력반...,바이폴러만의 프로세스로 사용되는 전력반도체는 대부분의 반도체 생산업체에서 제공하는 ...,original,[전체요약] 현재 바이폴러만의 프로세스(bipolar only process)로 사...
35997,한반도를 중심으로 벌어지고 있는 지정학적 대립의 관계가 그 어느 때 보다 빠르게 변...,"본 연구는 한국지리 교육과정 내용에 반영되어 있는 폐쇄적인 국토관을 비판하고, 관계...",original,[전체요약] 한반도를 중심으로 벌어지고 있는 지정학적 대립의 관계가 그 어느 때 보...
35998,공도회는 조선 후기 지방 유생의 학업진흥책이자 소과와 관련성이 높은 인재선발제도였다...,"조선 후기 공도회의 심층연구를 위한 작업으로, 18∼19세기 공도회의 운영 양상을 ...",original,[전체요약] 공도회는 조선 후기 지방 유생의 학업진흥책이자 소과와 관련성이 높은 인...


In [9]:
from datasets import Dataset

dataset = Dataset.from_pandas(merged_df);dataset

Dataset({
    features: ['original_text', 'summary_text', 'source_type', 'input_text'],
    num_rows: 36000
})

In [10]:
!git clone https://github.com/SKT-AI/KoBART #egg=kobart
%cd /kaggle/working/KoBART
os.getcwd()
from kobart import get_kobart_tokenizer,get_pytorch_kobart_model
#from transformers import BartForConditionalGeneration#,BartTokenizer
tokenizer = get_kobart_tokenizer()

Cloning into 'KoBART'...
remote: Enumerating objects: 334, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 334 (delta 15), reused 13 (delta 13), pack-reused 303 (from 1)
Receiving objects: 100% (334/334), 8.24 MiB | 15.25 MiB/s, done.
Resolving deltas: 100% (178/178), done.
/kaggle/working/KoBART
/kaggle/working/KoBART/.cache/kobart_base_tokenizer_cased_cf74400bce.zip[██████████████████████████████████████████████████]


In [11]:
def tokenize_function(examples):
    # 프롬프트가 붙은 input_text를 인코딩
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=512,
        padding="max_length",
        truncation=True
    )

    # 요약문도 target으로 인코딩
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary_text"],
            max_length=128,
            padding="max_length",
            truncation=True
        )["input_ids"]

    # pad token은 loss 계산에서 무시되도록 -100으로 변경
    labels = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in labels
    ]

    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [12]:
tokenized_dataset.save_to_disk("/kaggle/working/tokenized_val_datasets")

Saving the dataset (0/1 shards):   0%|          | 0/36000 [00:00<?, ? examples/s]

In [21]:
os.listdir('/kaggle/working')

['tokenized_val_datasets']